 <h1>20BCE020 <br>
Practical-6 IRS <br>
    Implement a program for document classification using Naive Bayes and Bernoulli. Compare the performance of both algorithms using suitable accuracy parameters. <br>
Burnoulli Bayes </h1>

In [1]:
import os
import pandas as pd
import numpy as np
from collections import defaultdict
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
# nltk.download("all")
from nltk.stem import PorterStemmer
stm=PorterStemmer()
n=4
stop_words = set(stopwords.words("english"))

In [2]:
fl=[]
for i in range(1,n+1):
    with open("N"+str(i)+".txt",'r') as mf:
        s=mf.read()
        s=word_tokenize(s)
        s=[w for w in s if w.casefold() not in stop_words]
        ll=[]
        for a in s:
            if a=="'" or a=="." or a==',' or a==":":
                continue
            else:
                ll.append(a)
        fl.append(ll)
print(fl)

[['Chinese', 'Beijing', 'Chinese'], ['Chinese', 'Chinese', 'Shanghai'], ['Chinese', 'Macao'], ['Tokyo', 'Japan', 'Chinese']]


In [3]:
for i in range(len(fl)):
    a=""
    for j in fl[i]:        
        a+=j+" "
    fl[i]=a

In [4]:
print(fl)

['Chinese Beijing Chinese ', 'Chinese Chinese Shanghai ', 'Chinese Macao ', 'Tokyo Japan Chinese ']


In [5]:
ll=['c','c','c','j']
unique=[]
classes=[]
class_p={}
class_prior={}
word_count={}

word_prob={}
for i in range(len(ll)):
    if ll[i] not in classes:
        classes.append(ll[i])
        class_p[ll[i]]=1
        word_count[ll[i]]={}
    else:
        class_p[ll[i]] += 1
        
for i in range(len(fl)):
    words=fl[i].split()
    for w in words:
        if w not in unique:
            unique.append(w)

for j in unique:
    for i in range(len(fl)):
        word_count.setdefault(ll[i], {}).setdefault(j, 0)
        if j in fl[i]:
            word_count[ll[i]][j] += 1
            

for c in classes:
    word_prob[c]={}
    for w in unique:
        word_prob[c][w]=(word_count[c][w]+1)/(class_p[c]+2)
        
total_docs=len(ll)
for c in classes:
    class_prior[c] = class_p[c]/len(ll)

In [6]:
print(classes)
print(class_prior)
print(word_count)
print(word_prob)

['c', 'j']
{'c': 0.75, 'j': 0.25}
{'c': {'Chinese': 3, 'Beijing': 1, 'Shanghai': 1, 'Macao': 1, 'Tokyo': 0, 'Japan': 0}, 'j': {'Chinese': 1, 'Beijing': 0, 'Shanghai': 0, 'Macao': 0, 'Tokyo': 1, 'Japan': 1}}
{'c': {'Chinese': 0.8, 'Beijing': 0.4, 'Shanghai': 0.4, 'Macao': 0.4, 'Tokyo': 0.2, 'Japan': 0.2}, 'j': {'Chinese': 0.6666666666666666, 'Beijing': 0.3333333333333333, 'Shanghai': 0.3333333333333333, 'Macao': 0.3333333333333333, 'Tokyo': 0.6666666666666666, 'Japan': 0.6666666666666666}}


In [7]:
test=[]
for i in range(1,3):
    with open("T"+str(i)+".txt",'r') as mf:
        s=mf.read()
        s=word_tokenize(s)
        s=[w for w in s if w.casefold() not in stop_words]
        ll=[]
        for a in s:
            if a=="'" or a=="." or a==',' or a==":":
                continue
            else:
                ll.append(a)
        test.append(ll)

In [8]:
print(test)

[['Chinese', 'Chinese', 'Chinese', 'Tokyo', 'Japan'], ['Chinese', 'Macao']]


In [9]:
for i in range(len(test)):
    a=""
    for j in test[i]:        
        a+=j+" "
    test[i]=a

In [10]:
print(test)

['Chinese Chinese Chinese Tokyo Japan ', 'Chinese Macao ']


In [11]:
unique_t=[]
for i in range(len(test)):
    words=test[i].split()
    unt=[]
    for w in words:
        if w not in unique_t:
            unt.append(w)
    unique_t.append(unt)

In [12]:
print(unique_t)

[['Chinese', 'Chinese', 'Chinese', 'Tokyo', 'Japan'], ['Chinese', 'Macao']]


In [13]:
lit=[]
for i in range(len(unique_t)):
    l=[]
    for x in unique:
        if x in unique_t[i]:
            l.append(1)
        else:
            l.append(0)
    lit.append(l)

In [14]:
print(lit)

[[1, 0, 0, 0, 1, 1], [1, 0, 0, 1, 0, 0]]


In [15]:
print(unique)

['Chinese', 'Beijing', 'Shanghai', 'Macao', 'Tokyo', 'Japan']


In [16]:
print(word_prob)

{'c': {'Chinese': 0.8, 'Beijing': 0.4, 'Shanghai': 0.4, 'Macao': 0.4, 'Tokyo': 0.2, 'Japan': 0.2}, 'j': {'Chinese': 0.6666666666666666, 'Beijing': 0.3333333333333333, 'Shanghai': 0.3333333333333333, 'Macao': 0.3333333333333333, 'Tokyo': 0.6666666666666666, 'Japan': 0.6666666666666666}}


In [17]:
pred=[]
pp=[]

for x in range(len(lit)):
    ap={}
    for c in classes:
        p=class_prior[c]
        for j in range(len(lit[x])):
            if lit[x][j]==1:
                p*=word_prob[c][unique[j]]
            if lit[x][j]==0:
                p*=(1-word_prob[c][unique[j]])           
            ap[c]=p
    pp.append(ap)

In [18]:
print(pp)
pred=[]
for d in pp:
    max_key = max(d, key=d.get)
    pred.append(max_key)
print(pred)

[{'c': 0.005184000000000001, 'j': 0.021947873799725653}, {'c': 0.05529600000000001, 'j': 0.0027434842249657075}]
['j', 'c']


In [19]:
y=['j','j']

In [20]:
from sklearn.metrics import confusion_matrix
import numpy as np
cmat=np.array(confusion_matrix(y,pred))
print("CONFUSION MATRIX:")
print(cmat)

print("PRECISION:")
print(np.trace(cmat)/np.sum(cmat))

CONFUSION MATRIX:
[[0 0]
 [1 1]]
PRECISION:
0.5


In [56]:
# pred=[]
# pp={}
# for c in classes:
#     max_prob = float('-inf')
#     ar_max=None
#     p=class_prior[c]
#     for x in range(len(lit)): 
#         if lit[x]==1:
#             p*=word_prob[c][unique[x]]
#         else:
#             p*=(1-word_prob[c][unique[x]])
#     if p > max_prob:
#         max_prob = p
#         arg_max = c
#     pp[c]=p
# pred.append(arg_max)

In [45]:
# pp={}
# for x in unique_t:
#     max_prob = float('-inf')
#     ar_max=None
#     for c in classes:
#         p=class_prior[c]
#         for j in unique:
#             if j in x:
#                 p*=word_prob[c][j]
#             else:
#                 p*=(1-word_prob[c][j])
#         pp[c]=p
                
            
        